In [2]:
# Import modules.
import os
import csv
import schedule
import pandas as pd
from os import path
from csv import writer
from googleapiclient.discovery import build
from dotenv import load_dotenv
from datetime import datetime

# Set API parameters. 
load_dotenv()
API_KEY = os.getenv("API_KEY")
CHANNEL_ID = os.getenv("CHANNEL_ID")
youtube = build('youtube', 'v3', developerKey = API_KEY)

# Primary function to get statistics.
class ytStatsBase:
    def __init__(self, API_KEY, CHANNEL_ID, youtube, channel_name):
        self.API_KEY = API_KEY
        self.CHANNEL_ID = CHANNEL_ID
        self.youtube = youtube

    # Get basic channel statistics.
    def get_channel_statistics(self):
        all_data = []
        request = youtube.channels().list(
            part = "snippet,contentDetails,statistics",
            id = CHANNEL_ID
            )
        response = request.execute()

        for i in range (len(response['items'])):
            data = dict(
                result_channel_name = response['items'][i]['snippet']['title'], 
                result_playlist_id = response['items'][i]['contentDetails']['relatedPlaylists']['uploads'],
                result_channel_subscribers = response['items'][i]['statistics']['subscriberCount'], 
                result_channel_views = response['items'][i]['statistics']['viewCount'], 
                result_channel_total_videos = response['items'][i]['statistics']['videoCount'] 
                )
            all_data.append(data)              
        return all_data

    # Get individual video IDs through playlist. 
    def get_video_ids(self):
        playlist_id = ytStatsBase.get_channel_statistics(youtube)[0]['result_playlist_id']
        
        request = youtube.playlistItems().list(
            part = 'contentDetails', 
            playlistId = playlist_id, 
            maxResults = 50
        )
        response = request.execute()

        all_data = []

        for i in range(len(response['items'])):
            all_data.append(response['items'][i]['contentDetails']['videoId'])

        next_page_token = response.get('nextPageToken')
        more_pages = True

        while more_pages:
            if next_page_token is None:
                more_pages = False
            else:
                request = youtube.playlistItems().list(
                    part = 'contentDetails', 
                    playlistId = playlist_id, 
                    maxResults = 50, 
                    pageToken = next_page_token
                )
                response = request.execute()
                
                for i in range(len(response['items'])):
                    all_data.append(response['items'][i]['contentDetails']['videoId'])
                
                next_page_token = response.get('nextPageToken')
        
        return all_data

    # Get individual video details. 
    def get_video_details(self):
        video_ids = ytStatsBase.get_video_ids(youtube)

        all_data = []
        for i in range(0, len(video_ids), 50):
            request = youtube.videos().list(
                part = 'snippet, statistics', 
                id = ','.join(video_ids[i: i + 50]),
                #order = 'date' # Not possible or a feature at this time, 12Dec22.
            )
            response = request.execute()

            for video in response['items']:
                video_stats = dict(
                    result_video_name = video['snippet']['title'],
                    result_video_id = video['id'],                
                    result_video_description = video['snippet']['description'],
                    result_video_upload_time = video['snippet']['publishedAt'],
                    result_video_views = video['statistics']['viewCount'],  
                    result_video_likes = video['statistics']['likeCount'],  
                    result_video_favorites = video['statistics']['favoriteCount'],  
                    result_video_comments = video['statistics']['commentCount']  
                    )
                all_data.append(video_stats)
        return all_data

    # Transfer data into dataframes.
    def create_data_frame(channel_stats, video_details):
        # Create data frames.
        initial_channel_data = pd.DataFrame(channel_stats, columns = [
            'result_playlist_id',
            'result_channel_name', 
            'result_channel_views',
            'result_channel_subscribers',
            'result_channel_total_videos' 
            ])
        initial_video_data = pd.DataFrame(video_details, columns = [
            'result_video_id',
            'result_video_upload_time', 
            'result_video_name', 
            'result_video_description', 
            'result_video_views', 
            'result_video_likes', 
            'result_video_comments' 
            ])
        return initial_channel_data, initial_video_data

    # Generate personalized files.
    def create_files(dfc, dfv):
        # Create base files. 
        isExist1, isExist2 = os.path.exists(f'{channel_name}(channel_data).csv'), os.path.exists(f'{channel_name}(video_data).csv')
        if isExist1 is True:
            pass
        elif isExist1 is False:
            dfc.to_csv(f'{channel_name}(channel_data).csv')
        if isExist2 is True:
            pass
        elif isExist2 is False:
            dfv.to_csv(f'{channel_name}(video_data).csv')

    # Gather new videos and append csv files.
    def append_new_videos_data(dfv):
        old_data = pd.read_csv(f'{channel_name}(video_data).csv', index_col=0)
        new_data = (pd.concat([old_data, dfv])
            .drop_duplicates(["result_video_id"], keep=False))
        df = pd.concat([new_data, old_data], ignore_index=True)
        df.to_csv(f'{channel_name}(video_data).csv')

    # Gather new videos and append csv files.
    def append_latest_statistics(dfv):
        initial_test_data = pd.DataFrame(dfv, columns = [
            'result_video_views',
            'result_video_likes', 
            'result_video_comments'
            ])
        initial_test_data.insert(0,'query_date',pd.to_datetime('today').strftime("%y/%m/%d"))
        df1 = initial_test_data
        df2  = pd.read_csv(f'{channel_name}(video_data).csv', index_col=0)
        df3 = pd.concat([df2, df1], axis=1)
        df3.to_csv(f'{channel_name}(video_data).csv')
        # return initial_test_data

# Call functions.
yt = ytStatsBase
channel_stats, video_details = yt.get_channel_statistics(youtube), yt.get_video_details(youtube)
channel_name = channel_stats[0]['result_channel_name']
dfc, dfv = yt.create_data_frame(channel_stats, video_details)
yt.create_files(dfc, dfv)
yt.append_new_videos_data(dfv)
yt.append_latest_statistics(dfv)